# Goalkeeper OLS analysis

In [1]:
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import MySQLdb

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score

### Connect DB & get Forword Player Data

In [3]:
db = MySQLdb.connect(
    "db.fastcamp.us",
    "root",
    "dkstncks",
    "football",
    charset='utf8',
)

def make_query(position):
    """
    
    parameter------------
    position : M, D, F, G
    
    return---------------
    SQL_QUERY String
    
    """
    SQL_QUERY = """
        SELECT 
            age, tall, weight, apps_start, apps_sub, mins, goals, assists, yel, red
            , spg, ps_x, motm, aw, tackles, inter, fouls, offsides, clear, drb, blocks
            , owng, keyp_x, fouled, off, disp, unstch, avgp, ps_y, rating
        FROM player
        WHERE position like "%{position}%" and mins > 270
        ;
    """.format(position=position)
    
    return SQL_QUERY

SQL_QUERY = make_query("G")
goalkeeper_df = pd.read_sql(SQL_QUERY, db)

len(goalkeeper_df)

213

### Scaling

In [4]:
X = goalkeeper_df.ix[:,:-1]
scaler = StandardScaler(with_mean=False)
X_scaled = scaler.fit_transform(X)

dfX0 = pd.DataFrame(X_scaled, columns=X.columns)
dfX = sm.add_constant(dfX0)
dfy = pd.DataFrame(goalkeeper_df.ix[:,-1], columns=["rating"])
g_df = pd.concat([dfX, dfy], axis=1)
g_df.head()

,age,tall,weight,apps_start,apps_sub,mins,goals,assists,yel,red,...,blocks,owng,keyp_x,fouled,off,disp,unstch,avgp,ps_y,rating
0,8.146320,41.924041,15.879078,3.099158,0.00000,3.031142,0.0,0.0,0.764448,2.85861,...,0.0,0.0,1.820579,0.936049,0.0,0.0,0.000000,6.940884,5.376996,6.62
1,5.353296,41.265549,15.149006,3.182919,0.00000,3.195594,0.0,0.0,0.000000,0.00000,...,0.0,0.0,0.000000,0.936049,0.0,0.0,1.736246,3.969252,4.490466,6.83
2,7.680816,40.387558,14.601451,2.680353,0.00000,2.689157,0.0,0.0,0.000000,0.00000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,5.667327,8.123310,6.92
3,5.586048,41.046051,15.514042,0.502566,2.12385,0.506436,0.0,0.0,0.000000,0.00000,...,0.0,0.0,1.820579,0.000000,0.0,0.0,0.000000,4.584804,8.036585,6.48
4,6.749808,39.290070,13.688861,2.512831,2.12385,2.546197,0.0,0.0,0.764448,0.00000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,5.752231,5.145727,6.48


### Summary OLS

In [5]:
model = sm.OLS(g_df.ix[:, -1], g_df.ix[:, :-1])
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                     6102.
Date:                Mon, 27 Jun 2016   Prob (F-statistic):          2.22e-259
Time:                        11:50:09   Log-Likelihood:                 6.8406
No. Observations:                 213   AIC:                             36.32
Df Residuals:                     188   BIC:                             120.4
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
age            0.0255      0.018      1.430      0.1

### Find Proper Model

In [7]:
# delete features based on t-value
remove_column_list = [
    "weight", "apps_start", "apps_sub", "mins", "goals", "assists", "yel", "red", "spg", "motm"
    , "tackles","inter", "fouls", "blocks", "owng", "keyp_x", "fouled", "off", "disp", "unstch"
]
removed_g_df = g_df.drop(remove_column_list, axis=1) 

model = sm.OLS(removed_g_df.ix[:, -1], removed_g_df.ix[:, :-1])
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 1.838e+04
Date:                Mon, 27 Jun 2016   Prob (F-statistic):          3.17e-284
Time:                        11:50:45   Log-Likelihood:                -20.978
No. Observations:                 213   AIC:                             55.96
Df Residuals:                     206   BIC:                             79.49
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
age            0.0267      0.018      1.452      0.1

In [9]:
# check F-value using anova table
formula_str = """
rating ~ age + tall + ps_x + aw + offsides + clear + drb +  avgp + ps_y
"""

model = sm.OLS.from_formula(formula_str, data=removed_g_df)
result = model.fit()
table_anova = sm.stats.anova_lm(result)
table_anova

,df,sum_sq,mean_sq,F,PR(>F)
age,1.0,0.004729,0.004729,0.086286,0.769250
tall,1.0,0.005905,0.005905,0.107752,0.743053
ps_x,1.0,0.400719,0.400719,7.311599,0.007427
aw,1.0,0.278175,0.278175,5.075640,0.025323
offsides,1.0,0.225671,0.225671,4.117637,0.043731
clear,1.0,0.042847,0.042847,0.781804,0.377626
drb,1.0,0.452490,0.452490,8.256233,0.004489
avgp,1.0,0.060146,0.060146,1.097446,0.296061
ps_y,1.0,0.035221,0.035221,0.642656,0.423679
Residual,205.0,11.235207,0.054806,NaN,NaN


In [10]:
# delete features based on F-value
remove_column_list = [
    "age", "offsides", "ps_y"
]
removed2_g_df = removed_g_df.drop(remove_column_list, axis=1) 

model = sm.OLS(removed2_g_df.ix[:, -1], removed2_g_df.ix[:, :-1])
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 2.133e+04
Date:                Mon, 27 Jun 2016   Prob (F-statistic):          5.94e-286
Time:                        11:51:27   Log-Likelihood:                -22.063
No. Observations:                 213   AIC:                             56.13
Df Residuals:                     207   BIC:                             76.29
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
tall           0.1423      0.004     40.454      0.0

### 결론
- 주요 변수 : tall, ps_x, dribble blocked